In [1]:
from IPython.display import display
import planner

controller = planner.PlanController("long-plan")
controller2 = planner.PlanController("plan")

display(controller.container)
display(controller2.container)

In [2]:
from IPython.display import display
import logger

import importlib
importlib.reload(logger)

log_controller = logger.LogController("logs")
display(log_controller.container)